In [2]:
from obspy import read
from obspy.io.sac.sactrace import SACTrace
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

RFDELYT = 5
LENGTH_RF_USE = 4

def DectectByIQR(filenames,ratio,attrname,value):
    amps=[]
    RFfilenames=[]
    for i,RFfile in enumerate(filenames):
        try:
            st = read(RFfile)
        except:
            continue
        if len(st)==0:
            continue
        tr = st[0]
        delta = tr.stats.delta
        pts1 = int(RFDELYT/delta)
        pts2 = int((RFDELYT+LENGTH_RF_USE)/delta)
        data = tr.data[pts1:pts2]
        if np.any(np.isnan(data)):
            continue
        RFfilenames.append(RFfile)
        amps.append(data)
    if len(RFfilenames)==0:
        return -1
    RFfilenames = np.array(RFfilenames)
    amps = np.array(amps)
    qs1,qs3 = np.quantile(amps,[0.25,0.75],axis=0)
    iqrs = qs3-qs1
    qs_top = qs3+ratio*iqrs
    qs_bottom = qs1-ratio*iqrs
    in_IGR = np.logical_and(np.greater(amps,qs_bottom),np.less(amps,qs_top))
    good = np.all(in_IGR,axis=1)
    RF_filenames_good = RFfilenames[good]
    number_all = len(RFfilenames)
    number_pick = len(RF_filenames_good)
    print("{0} of {1} events is selected by method of IQR with ratio of {2}".format(number_pick,number_all,ratio))
    print("now to change {0} of the selected sacfile to \'{1}\'".format(attrname,value))
    for filename in RF_filenames_good:
        try:
            sac = SACTrace.read(filename)
            setattr(sac,attrname,value)
            sac.write(filename) 
        except:
            continue   
    return 0


ratio = 1.5
attr_name = 'kt1'
Value = 'fool'
datadir= "../seismic_data_RFAC"
stations = glob("{0}/TASTE_*".format(datadir))
gausses = [2]
ispre=0
for station in stations:
    stationname = station.split("/")[-1]
    for gauss in gausses:
        filenames_group = glob(f"{station}/*.BHR.sac_g{gauss}".format(station,gauss))
        iii = DectectByIQR(filenames_group,ratio,attr_name,Value)
        if iii==-1:
            print("no data readed")






143 of 170 events is selected by method of IQR with ratio of 1.5
now to change kt1 of the selected sacfile to 'fool'
148 of 186 events is selected by method of IQR with ratio of 1.5
now to change kt1 of the selected sacfile to 'fool'
216 of 288 events is selected by method of IQR with ratio of 1.5
now to change kt1 of the selected sacfile to 'fool'
207 of 309 events is selected by method of IQR with ratio of 1.5
now to change kt1 of the selected sacfile to 'fool'
288 of 370 events is selected by method of IQR with ratio of 1.5
now to change kt1 of the selected sacfile to 'fool'
81 of 102 events is selected by method of IQR with ratio of 1.5
now to change kt1 of the selected sacfile to 'fool'
104 of 152 events is selected by method of IQR with ratio of 1.5
now to change kt1 of the selected sacfile to 'fool'
276 of 350 events is selected by method of IQR with ratio of 1.5
now to change kt1 of the selected sacfile to 'fool'
153 of 240 events is selected by method of IQR with ratio of 1.5


In [4]:
from obspy import read
from glob import glob
import shutil
import os

maindir = "../seismic_data_RFAC"
network = "TASTE"
stations = glob(f"{maindir}/{network}_*")
for station in stations:
    events_bad = []
    files = glob(f"{station}/*.BHR.sac_g2")
    for file in files:
        st = read(file)
        tr =st[0]
        if tr.stats.sac.get('kt1')!='fool':
            temp = f".{network}."
            events_bad.append(file.split("/")[-1].split(temp)[0])
    restoredir = f"{station}/bad"
    if not os.path.isdir(restoredir):
        os.makedirs(restoredir)
    for event in events_bad:
        files_remove = glob(f"{station}/{event}*")
        for file_remove in files_remove:
            try:
                shutil.move(file_remove,restoredir)
            except:
                continue





In [6]:
from obspy import read
from glob import glob
import shutil
import os

maindir = "../seismic_data_RFAC"
network = "TASTE"
stations = glob(f"{maindir}/{network}_*")
for station in stations:
    restoredir = f"{station}/bad"
 
    files = glob(f"{restoredir}/*.BHR.sac_g2")
    for file in files:
        eventname = file.split("/")[-1].split("BHR")[0]
        files_remove = glob(f"{station}/{eventname}*")
        for file_remove in files_remove:
            print(file_remove)
            try:
                shutil.move(file_remove,restoredir)
            except:
                continue


../seismic_data_RFAC/TASTE_T59/2018.362.13.03.33.TA.T59.BHR.sac
../seismic_data_RFAC/TASTE_T59/2018.362.13.03.33.TA.T59.BHZ.sac
../seismic_data_RFAC/TASTE_T59/2018.362.13.03.33.TA.T59.BHR.sac_g5
../seismic_data_RFAC/TASTE_T59/2018.362.13.03.33.TA.T59.BHR.sac_g7
../seismic_data_RFAC/TASTE_T59/2019.015.20.03.20.TA.T59.BHR.sac
../seismic_data_RFAC/TASTE_T59/2019.015.20.03.20.TA.T59.BHZ.sac
../seismic_data_RFAC/TASTE_T59/2019.015.20.03.20.TA.T59.BHR.sac_g5
../seismic_data_RFAC/TASTE_T59/2019.015.20.03.20.TA.T59.BHR.sac_g7
../seismic_data_RFAC/TASTE_T59/2019.262.07.32.01.TA.T59.BHR.sac
../seismic_data_RFAC/TASTE_T59/2019.262.07.32.01.TA.T59.BHZ.sac
../seismic_data_RFAC/TASTE_T59/2019.262.07.32.01.TA.T59.BHR.sac_g5
../seismic_data_RFAC/TASTE_T59/2019.262.07.32.01.TA.T59.BHR.sac_g7
../seismic_data_RFAC/TASTE_T59/2019.137.22.37.47.TA.T59.BHR.sac_g5
../seismic_data_RFAC/TASTE_T59/2019.137.22.37.47.TA.T59.BHR.sac_g7
../seismic_data_RFAC/TASTE_T59/2019.137.22.37.47.TA.T59.BHZ.sac
../seismic_data_

In [3]:
from obspy import read
from glob import glob
import shutil
import os

maindir = "seismic_data_RF"
network = "ZQ"
stations = glob(f"{maindir}/{network}_*")
for station in stations:
    events_bad = []
    files = glob(f"{station}/*.BHR.sac_g2")
    for file in files:
        st = read(file)
        tr =st[0]
        if np.all(tr.data==0) or np.any(np.isnan(tr.data)):
            temp = f".{network}."
            events_bad.append(file.split("/")[-1].split(temp)[0])
    restoredir = f"{station}/bad"
    if not os.path.isdir(restoredir):
        os.makedirs(restoredir)
    for event in events_bad:
        files_remove = glob(f"{station}/{event}*")
        for file_remove in files_remove:
            try:
                shutil.move(file_remove,restoredir)
            except:
                continue
